# Phys331 Lab Faraday Effect and Dispersion in Liquids

Here is the python package I used.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.optimize import curve_fit
%matplotlib inline
import matplotlib as mpl

1. Data preprocessing